In [1]:
import os
from dotenv import load_dotenv
from fredapi import Fred
import yfinance as yf
import pandas as pd
import numpy as np

Before running the cell below, you should create an account on [FRED's Website](https://fred.stlouisfed.org/docs/api/api_key.html) and create an API key. Store the key in a local .env file like this `FRED_API_KEY=your_actual_key_here_12345`. Note you do not need to run this file as all output data as been stored in raw_data.csv

In [2]:
# 1. SETUP
load_dotenv()
fred = Fred(api_key=os.getenv('FRED_API_KEY'))

In [5]:
# 2. FETCH MARKET DATA (YAHOO FINANCE)
tickers = ['^GSPC', '^VIX', 'SPY']
market_raw = yf.download(tickers, start='1993-01-29', progress=False)
market_data = pd.DataFrame({
    'GSPC': market_raw['Close']['^GSPC'],
    'VIX': market_raw['Close']['^VIX'],
    'SPY Volume': market_raw['Volume']['SPY']
})

# 3. FETCH MACRO DATA (FRED)
macro_data = pd.DataFrame({
    'GDP': fred.get_series('GDPC1'),
    'Core_Inflation': fred.get_series('PCEPILFE'),
    'Unemployment': fred.get_series('UNRATE'),
    'M2': fred.get_series('M2SL')
})

# Aligning the data to the same frequency (daily) and forward filling missing values
macro_data = macro_data.resample('D').ffill()

# 4. JOIN AND SAVE TO CSV
raw_data = market_data.join(macro_data, how='left')
raw_data = raw_data.ffill().dropna()
raw_data.to_csv('data/raw_data.csv')
print(f"Ingestion Complete. Shape: {raw_data.shape}. File: data/raw_data.csv")

Ingestion Complete. Shape: (8318, 7). File: data/raw_data.csv
